# Prerequisites

- RSA
- Coppersmith attacks

# Theory

Explanation https://youtu.be/RgbrpmJ49r4?t=2321

**Theorem** Howgrave-Graham  
Given a polynomial $f$ of degree $d$ and integer $N$, we can find in polynomial time roots $r$ modulo divisors $B$ of $N$ satisfying
$$f(r) \equiv 0 \ \bmod \ B$$ 
for $|B| > N^\beta$ when $|r| < N^{\beta^2 / d}$

Same idea as Coppersmith but we find a vector in a lattice less than $N^\beta < B$

**Theorem**:
    
Given half of the MSB / LSB bits of a factor $p$, we can factor and RSA modulus $N = pq$ in polynomial time

**Proof**:

Let $f(x) = x + a$ where $a$ are the MSB of $p$, and $r$ are the LSB of $p$, so $a+r = p$
- $f(x) \equiv 0 \ \bmod \ p > N^{1/2}$
- Apply Howgrave-Graham for $d = 1$ and $\beta = 1/2 \Rightarrow |r| < N^{\beta^2/d} = N^{1/4}$


We chose the polynomial basis:
$(x+a)^2, x+a, N$ which corresponds to the lattice
$$\begin{bmatrix}
R^2 & 2Ra & a^2 \\
0 &R &a \\
0 &0 &N
\end{bmatrix}
$$
with $\dim L = 3$ and  $\det L = R^3N$

From LLL => $|v| \approx \det L^{1/\dim L}$

The algorithm will find the root $r$ when $|g(r)| \leq |v| \approx \det L^{1/\dim L} <p $

# Code

## Toy implementation

In [1]:
p = random_prime(2^512)
q = random_prime(2^512)
N = p*q

In [2]:
#Delete 86 bits
a = p - (p % 1 << 86)
hex(a)

'0x2a208254dd7c2af458778606170548404b20e58ce4d42b62ab2c0bb67f1ccf2b744f23390be31df630c437fd0ef8bd921e2ae7335cd32d045f68353b44df10ad'

In [11]:
#Define the bound
R = 1 << 86 # 2 ^86

# The lattice matrix
M = matrix([
    [R^2, 2*R*a, a^2],
    [0, R, a],
    [0, 0, N]
])
R

77371252455336267181195264

In [6]:
# Get the desired basis using LLL
B = M.LLL()

In [13]:
# Form the polynomial using the first vector B[0]
Q = sum([B[0][i]* x^(2-i) / R^(2-i) for i in range(3)])
Q

-8983730112388468109345376900391223531774066957943554773713090848*x^2 + 416676948233725003993842292518121702864928552409734588110426667971366659548725479397154847*x

In [8]:
a + Q.roots(ring = ZZ)[0][0] == p

True

## Sage implementation

Sage has a function small_roots that applies coppersmith's theorem

Source for code: https://github.com/ashutosh1206/Crypton/tree/master/RSA-encryption/Attack-Coppersmith

In [21]:
p = random_prime(2^512)
q = random_prime(2^512)
N = p*q
#Delete 86 bits
a = p - (p% 1<<86)
hex(a)

'0xe76e46985dbab1c8f95d60f046c93eb6816a7f759f7b20bb351c9d3cc2448a553cee99dd7457fac0b079f1a61e26aee79e3342e61ff6a7fb7c43dacb5b229d67'

In [22]:
f = x + a

In [23]:
def factorization_n(f, N):
    P.<x> = PolynomialRing(Zmod(N))
    f = P(f)
    beta = .5
    dd = f.degree() # Degree of the polynomial (e for the above polynomial)
    epsilon = beta/7 #
    R = ceil(N**((beta**2/dd) - epsilon)) #The bound
    r = f.small_roots(R, beta, epsilon)
    return r

In [24]:
r = factorization_n(f, N)

In [25]:
a + r[0] == p

True

# Resources:

- [Coppersmith paper](https://cr.yp.to/bib/2001/coppersmith.pdf)
- LLL for RSA - A survey: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.9408&rep=rep1&type=pdf